# 1.Load Data & Libraries

In [ ]:
import numpy as np
import pandas as pd


from google.colab import drive
drive.mount("/gdrive")


import os
import pickle
os.chdir('/gdrive/MyDrive/LG 플젝-민/Sentiment Analysis 작업용/ABSA 최종df')
file_name = "ABSA_df.pkl" # sentiment analysis section에 업로드됨. 파일이름 동일.

with open(file_name,'rb') as fr:
    df = pickle.load(fr) 

Mounted at /gdrive


In [ ]:
df

,index,UniversalMessageId,SocialNetwork,source,CreatedTime,Domain,label,sents,customer_sents,remove_link_punc,...,fulfillment,happy,proud,superb,trust,sent,sentiment_dummy,neg,pos,sent_pred
0,0,REDDIT_267_rsvk9u,NaN,Reddit,Dec 31 14:54:03 2021,reddit.com,lg_ref,['Lg Model number: LMXS28626S\nImage: https://...,['lg model number: lmxs28626s\nimage: https://...,lg model number lmxs s image dispenser water k...,...,0,0,0,0,0,lg model number lmxs s image dispenser water k...,0,0.795137,-0.254036,0
1,2,WEB_98_2f02f04879eed91a7f5de5794099cc3c,NaN,Blog Post,Dec 31 14:34:00 2021,reidmuckr.fitnell.com,lg_ref,"[""The higher the standard of a filter, the gre...",['lg refrigerator water filters present high-q...,lg refrigerator water filters present high qua...,...,0,0,0,0,1,lg refrigerator water filters present high qua...,0,-3.611243,2.803974,1
2,4,WEB_100_sg_87167B4CEF951B83B2E73A5A8C79DF4B,NaN,Forums,Dec 31 14:21:00 2021,forums.redflagdeals.com,lg_ref,"[""bobcat99 wrote: ↑\nTwo people and you don't ...",['i previously had an lg washer that lasted fo...,i previously had an lg washer that lasted for ...,...,0,0,0,0,0,i previously had an lg washer that lasted for ...,0,-0.764530,0.976185,1
3,6,WEB_100_sg_87167B4CEF951B83B2E73A5A8C79DF4B,NaN,Forums,Dec 31 14:21:00 2021,forums.redflagdeals.com,lg_ref,"[""bobcat99 wrote: ↑\nTwo people and you don't ...",['i previously had an lg washer that lasted fo...,and based on my research it is not easy to cha...,...,0,0,0,0,0,and based on my research it is not easy to cha...,0,0.371561,0.147172,0
4,10,WEB_100_sg_60A151EA545F0EE99DFE2242188DD8AE,NaN,Forums,Dec 31 12:49:00 2021,forums.redflagdeals.com,lg_ref,"['I am in dire need of a dishwasher, fridge, f...","['i am in dire need of a dishwasher, fridge, f...",but i have heard so many terrible things about...,...,0,0,0,1,0,but i have heard so many terrible things about...,0,0.891424,-0.266681,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666205,1055768,YOUTUBE_44_UgxOHRzGKxG3cvjKhC94AaABAg,YOUTUBE,Youtube,2018-01-11 07:30:49,NaN,ss_ac,NaN,NaN,maybe samsung will also release smart closet o...,...,0,1,0,0,0,maybe samsung will also release smart closet o...,0,-1.788743,1.774704,1
666206,1055769,YOUTUBE_44_z22ns1ywrzv1xr45f04t1aokgjembf3jeur...,YOUTUBE,Youtube,2017-11-15 07:37:27,NaN,ss_ac,NaN,NaN,bobby mitsubishi are considered premium air c...,...,0,0,0,0,0,bobby mitsubishi are considered premium air c...,0,-0.274024,0.733706,1
666207,1055770,YOUTUBE_44_z22qwxbr5tfdifb0yacdp4345tbervze0wa...,YOUTUBE,Youtube,2017-10-18 03:50:33,NaN,ss_ac,NaN,NaN,you should go to vietnam not the cleanest stre...,...,0,0,0,0,0,you should go to vietnam not the cleanest stre...,0,2.146539,-1.078772,0
666208,1055772,YOUTUBE_44_z23vyj2wrzusy1phh04t1aokgqoaeas1wjf...,YOUTUBE,Youtube,2017-08-31 12:06:00,NaN,ss_ac,NaN,NaN,asher perlmuter ikr that s why i was amazed be...,...,0,1,0,1,0,asher perlmuter ikr that s why i was amazed be...,0,-2.415347,2.016587,1


## date 컬럼 처리


In [ ]:
# df['CreatedTime'].to_csv('created_time.csv') #
df['CreatedTime'] # 스프링클러 date format이 시간에 따라 달라서.. 치환할때 fixed format 주면안됨.

0         Dec 31 14:54:03 2021
1         Dec 31 14:34:00 2021
2         Dec 31 14:21:00 2021
3         Dec 31 14:21:00 2021
4         Dec 31 12:49:00 2021
                  ...         
666205     2018-01-11 07:30:49
666206     2017-11-15 07:37:27
666207     2017-10-18 03:50:33
666208     2017-08-31 12:06:00
666209     2017-07-26 06:26:47
Name: CreatedTime, Length: 666210, dtype: object

In [ ]:
df['dt'] = pd.to_datetime(df['CreatedTime'], errors='raise') # fixed format없이 dt형식으로 치환

In [ ]:
# def1 (시작날짜, 끝날짜, 회사) 주면 aspect 별 sentiment 값 나오게.
df['dt'] # 결측치 없음. 잘바뀜.

0        2021-12-31 14:54:03
1        2021-12-31 14:34:00
2        2021-12-31 14:21:00
3        2021-12-31 14:21:00
4        2021-12-31 12:49:00
                 ...        
666205   2018-01-11 07:30:49
666206   2017-11-15 07:37:27
666207   2017-10-18 03:50:33
666208   2017-08-31 12:06:00
666209   2017-07-26 06:26:47
Name: dt, Length: 666210, dtype: datetime64[ns]

In [ ]:
# 연월일을 각각 다른 컬럼으로. - 필요 없음. 더 복잡,오래걸림.
# df['year'] = df['dt'].dt.year
# df['month'] = df['dt'].dt.month
# df['day'] = df['dt'].dt.day

## 회사 구분 column 신설

In [ ]:
from tqdm import tqdm
tqdm.pandas()

lg = ['lg_ref','lg_wm','lg_ac']
ss = ['ss_ac','ss_ref', 'ss_wm']


def label_enterprise(label):
  if label in lg:
    return "lg"
  elif label in ss:
    return "ss"

df['company'] = df['label'].progress_apply(lambda label:label_enterprise(label))

100%|██████████| 666210/666210 [00:00<00:00, 834755.25it/s]


In [ ]:
df['company'].unique()

array(['lg', 'ss'], dtype=object)

In [ ]:
# product 나누기
from tqdm import tqdm
tqdm.pandas()

ref = ['lg_ref','ss_ref']
wm = ['ss_wm','lg_wm']
ac = ['lg_ac','ss_ac']


def label_enterprise(label):
  if label in ref:
    return "ref"
  elif label in wm:
    return "wm"
  elif label in ac:
    return "ac"

df['product'] = df['label'].progress_apply(lambda label:label_enterprise(label))

100%|██████████| 666210/666210 [00:00<00:00, 835670.45it/s]


In [ ]:
df['product'].unique()

array(['ref', 'wm', 'ac'], dtype=object)

In [ ]:
df['product'].value_counts()

ref    384909
wm     241090
ac      40211
Name: product, dtype: int64

# 2.Building Functions

## Aspect Based Sentiment Function

### basic score

input: 시작일, 끝일, 회사

output: 해당 기간ABSA 값

In [ ]:
# df for absa function
aspects = ['affordable', 'design', 'durable ', 'luxury', 'easy', 'performance','innovative', 'good_service ', 'fulfillment', 'happy', 'proud','superb', 'trust']

# (구)연월일 따로
# others = ['year', 'month', 'day','company'] # 구버젼
# absa_df = df[aspects + others]

# (신)연월일 한번에 --> better computing 시간 
absa_df = df[aspects + ['company','dt','sent_pred','product']]

# with open(file_name,'rb') as fr:
#     df = pickle.load(fr) 

In [ ]:
#pos/neg 구분용

# 바꿀 컬럼 이름들
neg_cols = []
for aspect in aspects:
  neg_cols.append("neg_"+aspect)
# print(neg_cols)

pos_cols = []
for aspect in aspects:
  pos_cols.append("pos_"+aspect)
# print(pos_cols)

In [ ]:
# sentiment 추가안됨.

def absa_info(start_date,end_date,company): # company는 "lg"아니면 "ss"
  temp_df = absa_df[absa_df['company'] == company] # 특정 company만
  temp_df = temp_df.drop('company',axis = 1)
  temp_df = temp_df[(pd.to_datetime(start_date) <= temp_df['dt']) & (temp_df['dt'] <= pd.to_datetime(end_date))] # 해당 date에 해당하는 data만 찾기
  temp_df = temp_df.drop('dt',axis = 1)
  # temp_df.sum() #는 해당 기간의 aspect count값. (pos/neg 구분 코드 skip하고 돌리면.)

  # aspect 별 sentiment 값 연산
  #positive/negative 컬럼df 만들기
  neg_df = temp_df[temp_df['sent_pred']==0]
  neg_df = neg_df[aspects]
  neg_df.columns = neg_cols
  neg_df



  pos_df = temp_df[temp_df['sent_pred']==1]
  pos_df = pos_df[aspects]
  pos_df.columns = pos_cols
  pos_df

  temp_df = pd.concat([neg_df,pos_df],axis = 1) 
  return temp_df.sum() # nan 자동 무시


start_date = '2019-01-01' # date 형식: 'YYYY-MM-DD'
end_date = '2019-12-31'
absa_info(start_date,end_date,'lg').to_csv('2019absa.csv') 

In [ ]:
absa_info(start_date,end_date,'lg')

neg_affordable        3052.0
neg_design            1026.0
neg_durable           7416.0
neg_luxury            1254.0
neg_easy               915.0
neg_performance       7480.0
neg_innovative         402.0
neg_good_service      8909.0
neg_fulfillment       4157.0
neg_happy             3221.0
neg_proud              514.0
neg_superb            5363.0
neg_trust             6614.0
pos_affordable        4693.0
pos_design            4710.0
pos_durable           3605.0
pos_luxury            2494.0
pos_easy              5034.0
pos_performance      23955.0
pos_innovative        1544.0
pos_good_service      5550.0
pos_fulfillment       2420.0
pos_happy            12568.0
pos_proud              354.0
pos_superb            9994.0
pos_trust             4016.0
dtype: float64

In [ ]:
df['sent'].iloc[1]

'lg refrigerator water filters present high quality drinking water by removing contaminants like pesticides chemicals and detergents '

### score by quarter

In [ ]:
# quarter 인식
def to_quarter(dt): # company는 "lg"아니면 "ss"
 return str(dt.year) + '_' + str(dt.quarter) + "Q" #2021_4Q

absa_df['quarter'] = absa_df['dt'].progress_apply(lambda create_dt: to_quarter(create_dt))
absa_df['quarter']
# absa_df.to_csv("quarter_df.csv")




100%|██████████| 666210/666210 [00:04<00:00, 139640.84it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0         2021_4Q
1         2021_4Q
2         2021_4Q
3         2021_4Q
4         2021_4Q
           ...   
666205    2018_1Q
666206    2017_4Q
666207    2017_4Q
666208    2017_3Q
666209    2017_3Q
Name: quarter, Length: 666210, dtype: object

In [ ]:
import pandas as pd
absa_df.to_csv('absa_df_quarter.csv')
# absa_df = pd.read_csv('absa_df_quarter.csv')

## jeeto challenge 날짜 제외

In [ ]:
def erase_specific_date_data(start_date,end_date):
  temp_df = absa_df
  temp_df = temp_df[(temp_df['dt'] < (pd.to_datetime(event_start_date)))|(( pd.to_datetime(event_end_date) < temp_df['dt']))]
  return temp_df
#2020.09.22~2020.10.15 jeeto.
event_start_date = '2020-09-22' # date 형식: 'YYYY-MM-DD'
event_end_date = '2020-10-15'

erase_specific_date_data(event_start_date,event_end_date)
temp_df.to_csv('no_jeeto.csv')

TypeError: ignored

In [ ]:
temp_df

,UniversalMessageId,SocialNetwork,SenderUserId,Title,Message,MessageType,CreatedTime,source
0,FACEBOOK_14_10152889869159377_10156974958924377,FACEBOOK,10153256326653866,NaN,I have a fridge similar to this one. Within ...,Facebook Comment,2017-12-31 14:57:38.000,Facebook
1,YOUTUBE_45_h0hF45peGOk,YOUTUBE,UCmYC3O8hab0555TaSghp9fw,[4K/60FPS]Let's Play Dirt 3: Complete Edition ...,!!!Extreme Lautstärke Warnung!!!\n\nNach extre...,YouTube Video,2017-12-31 13:28:35.000,Youtube
2,TWITTER_4_947446679036481538,TWITTER,2434425381,NaN,Hey @LGUS - great announcement at #CES - but t...,Twitter Mention,2017-12-31 12:37:45.000,Twitter
3,TWITTER_2_947430778941722624,TWITTER,3237275422,NaN,5 Benefits Of Using LG Instaview Refrigerator ...,Twitter Update,2017-12-31 11:34:34.000,Twitter
4,TWITTER_8_947430518013882368,TWITTER,1712763200,NaN,RT @LGIndia Uninterrupted Party with #LG Linea...,Twitter Retweet,2017-12-31 11:33:32.000,Twitter
...,...,...,...,...,...,...,...,...
86172,TWITTER_2_682847545148731392,TWITTER,3248730991,NaN,appliance repair lg refrigerator repair Tempe ...,Twitter Update,2016-01-01 08:55:16.114,Twitter
86173,TWITTER_2_682847543244533760,TWITTER,3248730991,NaN,appliance repair lg refrigerator repair Tempe ...,Twitter Update,2016-01-01 08:55:15.660,Twitter
86174,TWITTER_2_682847539436122112,TWITTER,3248730991,NaN,washer repair lg refrigerator repair Tempe AZ ...,Twitter Update,2016-01-01 08:55:14.752,Twitter
86175,TWITTER_7_682786753191657472,TWITTER,4686103580,NaN,@LG_SpaceLy my freezer is hotter than your mix...,Twitter Reply,2016-01-01 04:53:42.182,Twitter


# ABSA 값 뽑기

In [ ]:
# 해당 쿼터의 ABSA 값

def ABSAbyQ(year,quarter,company,product): # 2021,4,company(lg또는ss),product(ref,wm,ac)
  temp_df = absa_df[absa_df['company'] == company] # 특정 company만
  temp_df = temp_df.drop('company',axis = 1)

  temp_df = temp_df[temp_df['product'] == product] # 특정 product만
  temp_df = temp_df[temp_df['quarter'] == (str(year)+ "_" +str(quarter) + "Q")]

  # aspect 별 sentiment 값 연산
  #positive/negative 컬럼df 만들기
  neg_df = temp_df[temp_df['sent_pred']==0]
  neg_df = neg_df[aspects]
  neg_df.columns = neg_cols
  neg_df



  pos_df = temp_df[temp_df['sent_pred']==1]
  pos_df = pos_df[aspects]
  pos_df.columns = pos_cols
  pos_df

  temp_df = pd.concat([neg_df,pos_df],axis = 1) 
  return temp_df.sum() # nan 자동 무시 #

ABSAbyQ(2021,4,'lg','ac')

neg_affordable        15.0
neg_design             8.0
neg_durable           42.0
neg_luxury             6.0
neg_easy               6.0
neg_performance       73.0
neg_innovative        11.0
neg_good_service      60.0
neg_fulfillment       26.0
neg_happy             20.0
neg_proud              4.0
neg_superb            43.0
neg_trust             38.0
pos_affordable        68.0
pos_design            44.0
pos_durable           40.0
pos_luxury            30.0
pos_easy              55.0
pos_performance      264.0
pos_innovative        19.0
pos_good_service      94.0
pos_fulfillment       17.0
pos_happy            108.0
pos_proud              7.0
pos_superb           139.0
pos_trust             44.0
dtype: float64

In [ ]:
# 해당 쿼터의 ABSA 값

def ABSAbyQ(year,quarter,company,product): # 2021,4,company(lg또는ss),product(ref,wm,ac)
  temp_df = absa_df[absa_df['company'] == company] # 특정 company만
  temp_df = temp_df.drop('company',axis = 1)

  temp_df = absa_df[absa_df['product'] == product] # 특정 product만
  temp_df = temp_df[temp_df['quarter'] == (str(year)+ "_" +str(quarter) + "Q")]

  # aspect 별 sentiment 값 연산
  #positive/negative 컬럼df 만들기
  neg_df = temp_df[temp_df['sent_pred']==0]
  neg_df = neg_df[aspects]
  neg_df.columns = neg_cols
  neg_df



  pos_df = temp_df[temp_df['sent_pred']==1]
  pos_df = pos_df[aspects]
  pos_df.columns = pos_cols
  pos_df

  temp_df = pd.concat([neg_df,pos_df],axis = 1) 
  return temp_df.sum() # nan 자동 무시 #

ABSAbyQ(2021,4,'lg','ac')

neg_affordable        61.0
neg_design            27.0
neg_durable          128.0
neg_luxury            22.0
neg_easy              23.0
neg_performance      217.0
neg_innovative        25.0
neg_good_service     205.0
neg_fulfillment       85.0
neg_happy             59.0
neg_proud             13.0
neg_superb           121.0
neg_trust            151.0
pos_affordable       146.0
pos_design            79.0
pos_durable           94.0
pos_luxury            71.0
pos_easy              87.0
pos_performance      480.0
pos_innovative        56.0
pos_good_service     204.0
pos_fulfillment       33.0
pos_happy            209.0
pos_proud             15.0
pos_superb           234.0
pos_trust            109.0
dtype: float64